In [1]:
import mlflow

mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("DSPy")

<Experiment: artifact_location='mlflow-artifacts:/374362034103955121', creation_time=1741686562632, experiment_id='374362034103955121', last_update_time=1741686562632, lifecycle_stage='active', name='DSPy', tags={}>

In [2]:
mlflow.dspy.autolog()

/Users/aidand/dev/hello-dspy/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import dspy

lm = dspy.LM('fireworks_ai/accounts/fireworks/models/llama-v3p1-8b-instruct', max_tokens=3000)
gpt4o = dspy.LM('openai/gpt-4o', max_tokens=3000)

dspy.configure(lm=lm)

In [ ]:
%pip install -U bm25s PyStemmer "jax[cpu]"

In [2]:
from dspy.utils import download

download("https://huggingface.co/dspy/cache/resolve/main/wiki.abstracts.2017.tar.gz")
!tar -xzvf wiki.abstracts.2017.tar.gz

x wiki.abstracts.2017.jsonl


In [4]:
import ujson
corpus = []

with open("wiki.abstracts.2017.jsonl") as f:
    for line in f:
        line = ujson.loads(line)
        corpus.append(f"{line['title']} | {' '.join(line['text'])}")

len(corpus)

5233330

In [5]:
import bm25s
import Stemmer

stemmer = Stemmer.Stemmer("english")
corpus_tokens = bm25s.tokenize(corpus, stopwords="en", stemmer=stemmer)

retriever = bm25s.BM25(k1=0.9, b=0.4)
retriever.index(corpus_tokens)

In [6]:
import random
from dspy.datasets import DataLoader

kwargs = dict(fields=("claim", "supporting_facts", "hpqa_id", "num_hops"), input_keys=("claim",))
hover = DataLoader().from_huggingface(dataset_name="hover-nlp/hover", split="train", trust_remote_code=True, **kwargs)

hpqa_ids = set()
hover = [
    dspy.Example(claim=x.claim, titles=list(set([y["key"] for y in x.supporting_facts]))).with_inputs("claim")
    for x in hover
    if x["num_hops"] == 3 and x["hpqa_id"] not in hpqa_ids and not hpqa_ids.add(x["hpqa_id"])
]

random.Random(0).shuffle(hover)
trainset, devset, testset = hover[:200], hover[200:500], hover[650:]

In [7]:
example = trainset[0]

print("Claim:", example.claim)
print("Pages that must be retrieved:", example.titles)

Claim: This director is known for his work on Miss Potter. The Academy of Motion Picture Arts and Sciences presents the award in which he was nominated for his work in "Babe".
Pages that must be retrieved: ['Chris Noonan', 'Academy Award for Best Director', 'Miss Potter']


In [8]:
def search(query: str, k: int) -> list[str]:
    tokens = bm25s.tokenize(query, stopwords="en", stemmer=stemmer, show_progress=False)
    results, scores = retriever.retrieve(tokens, k=k, n_threads=1, show_progress=False)
    run = {corpus[doc]: float(score) for doc, score in zip(results[0], scores[0])}
    return run

In [9]:
class Hop(dspy.Module):
    def __init__(self, num_docs=10, num_hops=4):
        self.num_docs, self.num_hops = num_docs, num_hops
        self.generate_query = dspy.ChainOfThought('claim, notes -> query')
        self.append_notes = dspy.ChainOfThought('claim, notes, context -> new_notes: list[str], titles: list[str]')

    def forward(self, claim: str) -> list[str]:
        notes = []
        titles = []

        for _ in range(self.num_hops):
            query = self.generate_query(claim=claim, notes=notes).query
            context = search(query, k=self.num_docs)
            prediction = self.append_notes(claim=claim, notes=notes, context=context)
            notes.extend(prediction.new_notes)
            titles.extend(prediction.titles)
        
        return dspy.Prediction(notes=notes, titles=list(set(titles)))

In [10]:
def top5_recall(example, pred, trace=None):
    gold_titles = example.titles
    recall = sum(x in pred.titles[:5] for x in gold_titles) / len(gold_titles)

    # If we're "bootstrapping" for optimization, return True if and only if the recall is perfect.
    if trace is not None:
        return recall >= 1.0
    
    # If we're just doing inference, just measure the recall.
    return recall

evaluate = dspy.Evaluate(devset=devset, metric=top5_recall, num_threads=16, display_progress=True, display_table=5)

In [11]:
evaluate(Hop())

Average Metric: 16.67 / 71 (23.5%):  23%|██▎       | 70/300 [02:13<09:15,  2.42s/it]

2025/03/13 20:04:48 ERROR dspy.utils.parallelizer: Error for Example({'claim': 'The band that collaborated with Neva Dinova on the album "One Jug of Wine, Two Vessels", and Disturbed are both rock bands from America.', 'titles': ['Bright Eyes (band)', 'Disturbed (band)', 'One Jug of Wine, Two Vessels']}) (input_keys={'claim'}): Expected dict_keys(['reasoning', 'new_notes', 'titles']) but got dict_keys(['reasoning', 'new_notes']). Set `provide_traceback=True` for traceback.


Average Metric: 82.33 / 299 (27.5%): 100%|██████████| 300/300 [08:32<00:00,  1.71s/it]

2025/03/13 20:11:05 INFO dspy.evaluate.evaluate: Average Metric: 82.33333333333334 / 300 (27.4%)


,claim,example_titles,notes,pred_titles,top5_recall,titles
0,Nike football team has had a player endorse the football boot Nike...,"[Nike Total 90, Marcus Rashford, Nike Hypervenom]",['The Nike Total 90 range has now been replaced with Nike Hyperven...,['2011 AFC Cup | The 2011 AFC Cup was the 8th edition of the AFC C...,✔️ [0.333],NaN
1,Bill Boyd is the chairman of the appliance company that operates t...,"[Suncoast Hotel and Casino, Thomas Eje, Boyd Gaming]",['The claim does not match any of the context information provided...,"['Portrait of an Unknown Woman', 'The Unknown Warrior', 'Frederick...",,NaN
2,The president of South Korea was born 24 January 1953. The group t...,"[Moon Jae-in, Euh Yoon-dae, Presidential Council on Nation Brandin...","['The president of South Korea was born in 1924, not 1953.', 'The ...","['Euh Yoon-Dae', '2000 inter-Korean summit', 'President of South K...",,NaN
3,The movie Khan Kluay was released 2 months before the 2009 movie t...,"[Fantastic Mr. Fox (film), Khan Kluay, Jason Schwartzman]","['The movie Khan Kluay was released in 2006', 'Jason Schwartzman c...","['Fantastic Mr Fox (opera)', 'Khan Kluay', 'List of Khan Kluay cha...",✔️ [0.333],NaN
4,The director of Finding Dory co-directed the film A Bug's Life.,"[Andrew Stanton, Finding Dory, A Bug's Life]","['The director of Finding Dory is Andrew Stanton.', ""Andrew Stanto...","[Finding Dory, Co-director of A Bug's Life, Incorrect Claim, Andre...",✔️ [0.667],NaN


27.44

[Trace(request_id=80407866ba0a4527b103664a5bda1e13), Trace(request_id=0051ffcd06b7490b87e96d1e785cbb50), Trace(request_id=977a0578e1024e5e9bc541f3d8c5393d), Trace(request_id=4052462bd5a545c9b6e9a9c9fbb30ac5), Trace(request_id=755c1fce57e24f338c9ec9214c9e98a4), Trace(request_id=f025604a97cc46009283a9d655dcf19f), Trace(request_id=d9f5221d6f4c45e7a2c5be0e9d053bd7), Trace(request_id=3542bb9b92374d758262c25ca61751c2), Trace(request_id=29d887d6b9ce4706ad9002c77c68c1ab), Trace(request_id=e6f7e29a7abb437187603906d0f5b874)]

In [12]:
models = dict(prompt_model=gpt4o, teacher_settings=dict(lm=gpt4o))
tp = dspy.MIPROv2(metric=top5_recall, auto="medium", num_threads=16, **models)

kwargs = dict(minibatch_size=40, minibatch_full_eval_steps=4, requires_permission_to_run=False)
optimized = tp.compile(Hop(), trainset=trainset, max_bootstrapped_demos=4, max_labeled_demos=4, **kwargs)

2025/03/13 20:11:19 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING MEDIUM AUTO RUN SETTINGS:
num_trials: 25
minibatch: True
num_candidates: 9
valset size: 160

2025/03/13 20:11:19 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/03/13 20:11:19 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.

2025/03/13 20:11:19 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=9 sets of demonstrations...


Bootstrapping set 1/9
Bootstrapping set 2/9
Bootstrapping set 3/9


 40%|████      | 16/40 [09:15<13:53, 34.73s/it]


Bootstrapped 4 full traces after 16 examples for up to 1 rounds, amounting to 16 attempts.
Bootstrapping set 4/9


  2%|▎         | 1/40 [00:38<25:08, 38.68s/it]


...

2025/03/13 21:35:54 INFO dspy.evaluate.evaluate: Average Metric: 25.000000000000004 / 40 (62.5%)
2025/03/13 21:35:54 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 62.5 on minibatch of size 40 with parameters ['Predictor 0: Instruction 0', 'Predictor 0: Few-Shot Set 6', 'Predictor 1: Instruction 7', 'Predictor 1: Few-Shot Set 3'].
2025/03/13 21:35:54 INFO dspy.teleprompt.mipro_optimizer_v2: Minibatch scores so far: [64.17, 54.17, 65.83, 51.67, 50.83, 67.5, 62.5, 68.33, 67.5, 59.17, 64.17, 54.17, 57.5, 54.17, 62.5, 55.83, 60.0, 65.0, 61.67, 64.17, 57.5, 28.33, 62.5]
2025/03/13 21:35:54 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [29.58, 62.5, 62.29, 64.79, 65.0, 62.08, 59.79, 63.54]
2025/03/13 21:35:54 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 65.0
2025/03/13 21:35:54 INFO dspy.teleprompt.mipro_optimizer_v2: ==========================================


2025/03/13 21:35:54 INFO dspy.teleprompt.mipro_optimizer_v2: ===== Trial 32 / 32 - Full Ev


Average Metric: 97.00 / 160 (60.6%): 100%|██████████| 160/160 [03:24<00:00,  1.28s/it]

2025/03/13 21:39:19 INFO dspy.evaluate.evaluate: Average Metric: 97.00000000000001 / 160 (60.6%)
2025/03/13 21:39:19 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [29.58, 62.5, 62.29, 64.79, 65.0, 62.08, 59.79, 63.54, 60.63]
2025/03/13 21:39:19 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 65.0
2025/03/13 21:39:19 INFO dspy.teleprompt.mipro_optimizer_v2: =======================
2025/03/13 21:39:19 INFO dspy.teleprompt.mipro_optimizer_v2: 

2025/03/13 21:39:19 INFO dspy.teleprompt.mipro_optimizer_v2: == Trial 33 / 32 - Minibatch ==



Average Metric: 25.00 / 40 (62.5%): 100%|██████████| 40/40 [00:42<00:00,  1.06s/it]

2025/03/13 21:40:01 INFO dspy.evaluate.evaluate: Average Metric: 24.999999999999996 / 40 (62.5%)
2025/03/13 21:40:01 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 62.5 on minibatch of size 40 with parameters ['Predictor 0: Instruction 8', 'Predictor 0: Few-Shot Set 2', 'Predictor 1: Instruction 5', 'Predictor 1: Few-Shot Set 2'].
2025/03/13 21:40:01 INFO dspy.teleprompt.mipro_optimizer_v2: Minibatch scores so far: [64.17, 54.17, 65.83, 51.67, 50.83, 67.5, 62.5, 68.33, 67.5, 59.17, 64.17, 54.17, 57.5, 54.17, 62.5, 55.83, 60.0, 65.0, 61.67, 64.17, 57.5, 28.33, 62.5, 62.5]
2025/03/13 21:40:01 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [29.58, 62.5, 62.29, 64.79, 65.0, 62.08, 59.79, 63.54, 60.63]
2025/03/13 21:40:01 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 65.0
2025/03/13 21:40:01 INFO dspy.teleprompt.mipro_optimizer_v2: ==========================================


2025/03/13 21:40:01 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best i

In [14]:
optimized.save("optimized_multihop.json")

In [15]:
evaluate(optimized)

Average Metric: 201.67 / 300 (67.2%): 100%|██████████| 300/300 [08:34<00:00,  1.71s/it]

2025/03/13 21:51:32 INFO dspy.evaluate.evaluate: Average Metric: 201.66666666666657 / 300 (67.2%)


,claim,example_titles,notes,pred_titles,top5_recall
0,Nike football team has had a player endorse the football boot Nike...,"[Nike Total 90, Marcus Rashford, Nike Hypervenom]","['Nike Total 90 was replaced by Nike Hypervenom.', 'Nike Hyperveno...","[Robert Lewandowski, Kylian Mbappé, Nike Total 90, Marcus Rashford...",✔️ [1.000]
1,Bill Boyd is the chairman of the appliance company that operates t...,"[Suncoast Hotel and Casino, Thomas Eje, Boyd Gaming]",['Bill Boyd is the executive chairman of Boyd Gaming Corporation.'...,"[Boyd Gaming Corporation, Thomas Eje, Suncoast Hotel and Casino, B...",✔️ [0.667]
2,The president of South Korea was born 24 January 1953. The group t...,"[Moon Jae-in, Euh Yoon-dae, Presidential Council on Nation Brandin...","['The president of South Korea was born on January 24, 1953.', 'Eu...","[Euh Yoon-Dae, President of South Korea, Moon Jae-in, KB Financial...",✔️ [0.667]
3,The movie Khan Kluay was released 2 months before the 2009 movie t...,"[Fantastic Mr. Fox (film), Khan Kluay, Jason Schwartzman]","['Khan Kluay was released in 2006.', 'Jason Schwartzman collaborat...","[Fantastic Mr. Fox, Khan Kluay, Jason Schwartzman]",✔️ [0.667]
4,The director of Finding Dory co-directed the film A Bug's Life.,"[Andrew Stanton, Finding Dory, A Bug's Life]","['Andrew Stanton directed Finding Dory.', ""Andrew Stanton co-direc...","[Andrew Stanton, Lee Unkrich, Finding Dory, A Bug's Life]",✔️ [1.000]


67.22

[Trace(request_id=7bbdc38de8c444cc88513cf28ad79e47), Trace(request_id=8129bcf4d3e146e3bb001dc6f855b901), Trace(request_id=aba3e7e6bc5a44cca70afd305d4f811e), Trace(request_id=e82e31eb46fa4c5f9eb30afa4cca24ee), Trace(request_id=844bc9b9844e47099da750af79dd8b7c), Trace(request_id=e4d9a179e74f4a14abf6899574a96453), Trace(request_id=555b36a23d874e70beb76f3ee1473d98), Trace(request_id=0f219fc1b9d8419f9213dd07201ce179), Trace(request_id=b75defb0113e43e49307943a1c9e0a19), Trace(request_id=57b5a3ce5b824b2f8ce48f0535d971e4)]

In [19]:
Hop()(claim="The author of the 1960s unproduced script written for The Beatles, Up Against It, and Bernard-Marie Koltès are both playwrights.").titles

['Tabataba suivi de pawana',
 'Up Against It!',
 'The Alien',
 'The Illusionist',
 'Bernard-Marie Koltès',
 'In the Solitude of Cotton Fields',
 'Faultless disagreement',
 'Up Against It',
 'The Hook',
 "The Kilroys' List"]

Trace(request_id=754456d5e931401eb0a60c4d1b3b46ca)

In [20]:
optimized(claim="The author of the 1960s unproduced script written for The Beatles, Up Against It, and Bernard-Marie Koltès are both playwrights.").titles

['Joe Orton',
 'Bernard-Marie Koltès',
 'What the Butler Saw (play)',
 'Up Against It']

Trace(request_id=9d0cf60b8ab447d9908223552a7aea83)

In [21]:
dspy.inspect_history(n=1)





[2025-03-13T21:59:46.192830]

System message:

Your input fields are:
1. `claim` (str)
2. `notes` (str)
3. `context` (str)

Your output fields are:
1. `reasoning` (str)
2. `new_notes` (list[str])
3. `titles` (list[str])

All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## claim ## ]]
{claim}

[[ ## notes ## ]]
{notes}

[[ ## context ## ]]
{context}

[[ ## reasoning ## ]]
{reasoning}

[[ ## new_notes ## ]]
{new_notes}        # note: the value you produce must adhere to the JSON schema: {"type": "array", "items": {"type": "string"}}

[[ ## titles ## ]]
{titles}        # note: the value you produce must adhere to the JSON schema: {"type": "array", "items": {"type": "string"}}

[[ ## completed ## ]]

In adhering to this structure, your objective is: 
        Using the provided `claim`, `notes`, and `context`, synthesize the information to generate `new_notes` and `titles`. The `new_notes` should include refined and relevant insights t